In [ ]:
from selenium.webdriver import Chrome
import pickle
import requests
from functools import partial
import html

import re
import string
import nltk
import pymystem3
from bs4 import BeautifulSoup
from nltk.corpus import stopwords

In [2]:
x = Chrome()

WebDriverException: Message: Could not create a session: You must enable the 'Allow Remote Automation' option in Safari's Develop menu to control Safari via WebDriver.


In [14]:
url = ("https://www.google.ru/search?"
"q=site:{site}{params}&newwindow=1&tbs=cdr:1,"
"cd_min:{min_month}/{min_day}/{min_year},cd_max:{max_month}/{max_day}/{max_year},sbd:1")

In [15]:
def url_gen(min_date, max_date, site, params=''):
    d_min = min_date.split('.')
    d_max = max_date.split('.')
    return url.format(site=site,
                      params=params,
                      min_month=d_min[1],min_day=d_min[0],min_year=d_min[2],
                      max_month=d_max[1],max_day=d_max[0],max_year=d_max[2],
                     )

In [16]:
forbes = partial(url_gen, site='forbes.ru', params='+news')
bankir_ru = partial(url_gen, site='bankir.ru', params='/novosti/')
arb = partial(url_gen, site='arb.ru/b2b/news/')
bosfera = partial(url_gen, site='bosfera.ru/press-release/')

dates = [('1.4.2017', '30.4.2017'), ('1.5.2017', '31.5.2017'), ('1.6.2017', '30.6.2017')]

url_list = [
    [forbes(d[0], d[1]) for d in dates],
    [bankir_ru(d[0], d[1]) for d in dates],
    [arb(d[0], d[1]) for d in dates],
    [bosfera(d[0], d[1]) for d in dates]
]

In [17]:
# collecting links from google
links = []
for site in url_list:
    for url in site:
        x.get(url)
        while True:
            h3 = x.find_elements_by_css_selector('.r')
            st = x.find_elements_by_css_selector('.g .s .st .f')
            for elem, date in zip(h3, st):
                link = (elem.find_element_by_tag_name('a').get_property("href"))
                links.append({'date': date.text[:-6], 'link': link})
            try:
                next_link = x.find_element_by_id('pnnext').get_property("href")
                x.get(next_link)
            except:
                break
    with open('links.pickle', 'wb+') as f:
        pickle.dump(file=f, obj=links)

In [10]:
# saving data
with open('links.pickle', 'wb+') as f:
    pickle.dump(file=f, obj=links)

In [7]:
def collect_from_bosfera(news, x):
    news['title'] = x.find_element_by_css_selector('h1').text
    news['text'] = '\n'.join([html.unescape(t.text) for t in x.find_elements_by_css_selector('.inart p')])                                                    
    return news

def collect_from_arb(news, x):
    news['title'] = x.find_element_by_css_selector('h1').text
    news['text'] = '\n'.join([html.unescape(t.text) for t in x.find_elements_by_css_selector('.editorBox p')])
    news['datetime'] = x.find_element_by_css_selector('.date').text
    news['section'] = news['datetime'].split('\\')[1]
    news['datetime'] = news['datetime'].split('\\')[0]
    return news

def collect_from_forbes(news, x):
    if news['link'].split('/')[3] in ('profile', 'biznes-video', 'person', 'milliardery-video'):
        return news
    news['title'] = x.find_element_by_css_selector('h1').text
    news['text'] = html.unescape(x.find_element_by_css_selector('.body-page').text)
    
    news['datetime'] = x.find_element_by_css_selector('.date').text
    news['section'] = x.find_element_by_css_selector('.section').text
    
    
    try:
        news['hashtag'] = x.find_element_by_css_selector('.hashtag').text
    except:
        pass
    try:
        news['subtitle'] = x.find_element_by_css_selector('.subtitle').text
    except:
        pass
    
    return news

def collect_from_bankir(news, x):
    news['title'] = x.find_element_by_css_selector('h1').text
    news['text'] = html.unescape(x.find_element_by_css_selector('.article-content').text)
    news['section'] = ','.join([e.text for e in x.find_elements_by_css_selector('li a .toggle-item')])
    news['datetime'] = x.find_element_by_css_selector('.article-date').text

    try:
        news['hashtag'] = x.find_element_by_css_selector('.article-tags').text
    except:
        pass
    try:
        news['subtitle'] = x.find_element_by_css_selector('.h2').text
    except:
        pass
    
    return news

In [8]:
# dictionary of functions for switch/case
site_processer = {
    'www.forbes.ru' : collect_from_forbes,
    'bankir.ru' : collect_from_bankir,
    'arb.ru' : collect_from_arb,
    'bosfera.ru': collect_from_bosfera
}

In [9]:
# collecting news
goal = len(links) + 1
for news, i in zip(links, range(1, goal)):
    if 'title' in news.keys():
        continue
    
    try:
        x.get(news['link'])
        news = site_processer[news['link'].split('/')[2]](news, x)
    except:
        pass

    if i % 5 == 0:
        print("Progress %.2f %%" % (i / goal * 100))

In [ ]:
# saving data
with open('news.pickle', 'wb+') as f:
    pickle.dump(file=f, obj=objlist)

In [ ]:
def clean_text(text):
    text = re.sub(' +|\n*', ' ', regex.sub(BeautifulSoup(text,"lxml").get_text(), ' ')).lstrip(' ').rstrip(' ')
    text = ' '.join([e for e in mystem.lemmatize(text) if e not in stopwords.words("russian")])
    return text

In [ ]:
mystem = pymystem3.Mystem()
regex = re.compile('|'.join(string.punctuation+"«»€№"))

In [ ]:
for news, i in zip(links, range(1, len(links) + 1)):
    if 'text' in news.keys():
        news['text'] = clean_text(news['text'])
        cleared.append(d)
    if i % 5 == 0:
        print("Progress %.2f %%" % (i / goal * 100))

In [ ]:
with open("all_news_clean.pickle","wb") as f:
    pickle.dump(links, f)